#### Milestone 2 #####

In [32]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import json


Load dataframes into proper format

In [84]:
df_plot = pd.read_csv("data/MovieSummaries/plot_summaries.txt", sep='\t', header=None, names=["wikiID", "plot"])
df_meta = pd.read_csv("data/MovieSummaries/movie.metadata.tsv", sep='\t', header=None, names=["wikiID", "freeID", "name", "release_date", "revenue", "runtime", "languages", "countries", "genres"])
df_char = pd.read_csv("data/MovieSummaries/character.metadata.tsv", sep='\t',header=None, names=["WikiID","freeID","char_name","actor_DOB","actor_gender","actor_height","actor_ethnicity","actor_name","Actor_age","freeID_char_map", "FreeID_char", "FreeID_actor"])
df_char_names = pd.read_csv('data/MovieSummaries/name.clusters.txt', sep="\t", header=None, names=["char_name","freeID_char_map"])

In [87]:
from pandas.io.json import json_normalize

df_tropes = pd.read_csv('data/MovieSummaries/tvtropes.clusters.txt', sep='\t', header=None, names=["trope","details"])
#flatten json column into panda dataframe:
df_tropes = df_tropes.join(json_normalize(df_tropes["details"].map(json.loads).tolist())).drop(["details"], axis=1).rename(columns={"id":"freeID_char_map"})

C:\Users\YGrebici\AppData\Local\Temp\ipykernel_64688\100173281.py:5: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead.
  df_tropes = df_tropes.join(json_normalize(df_tropes["details"].map(json.loads).tolist())).drop(["details"], axis=1).rename(columns={"id":"freeID_char_map"})


In [88]:
df_tropes.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 501 entries, 0 to 500
Data columns (total 5 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   trope            501 non-null    object
 1   char             501 non-null    object
 2   movie            501 non-null    object
 3   freeID_char_map  501 non-null    object
 4   actor            501 non-null    object
dtypes: object(5)
memory usage: 19.7+ KB


In [ ]:
df_meta.info()

In [ ]:
df_plot.info()

In [81]:
df_char.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 450669 entries, 975900 to 28308153
Data columns (total 12 columns):
 #   Column           Non-Null Count   Dtype  
---  ------           --------------   -----  
 0   WikiID           450669 non-null  object 
 1   freeID           440674 non-null  object 
 2   char_name        192794 non-null  object 
 3   actor_DOB        344524 non-null  object 
 4   actor_gender     405060 non-null  object 
 5   actor_height     154824 non-null  float64
 6   actor_ethnicity  106058 non-null  object 
 7   actor_name       449441 non-null  object 
 8   Actor_age        292556 non-null  float64
 9   freeID_char_map  450669 non-null  object 
 10  FreeID_char      192804 non-null  object 
 11  FreeID_actor     449854 non-null  object 
dtypes: float64(2), object(10)
memory usage: 44.7+ MB


In [76]:
df_tropes.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 501 entries, 0 to 500
Data columns (total 5 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   trope    501 non-null    object
 1   t.char   501 non-null    object
 2   t.movie  501 non-null    object
 3   t.id     501 non-null    object
 4   t.actor  501 non-null    object
dtypes: object(5)
memory usage: 19.7+ KB


In [82]:
df_char_names.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2666 entries, 0 to 2665
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   0       2666 non-null   object
 1   1       2666 non-null   object
dtypes: object(2)
memory usage: 41.8+ KB


The feature connecting dataframes together is the Wikipedia ID. Also there are more metadatas of movies (81741 movies) than plots (42303 movies). We will only keep the metadatas of the movies we know the plot of.

In [71]:
df_full = df_meta.merge(df_plot, how='inner', on="wikiID")
df_full.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 42204 entries, 0 to 42203
Data columns (total 10 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   wikiID        42204 non-null  int64  
 1   freeID        42204 non-null  object 
 2   name          42204 non-null  object 
 3   release_date  39586 non-null  object 
 4   revenue       7587 non-null   float64
 5   runtime       35580 non-null  float64
 6   languages     42204 non-null  object 
 7   countries     42204 non-null  object 
 8   genres        42204 non-null  object 
 9   plot          42204 non-null  object 
dtypes: float64(2), int64(1), object(7)
memory usage: 3.5+ MB


Only a small fraction of the characters have been labellised with a trope (500), compared to the number of unlabeled (450669). Here is the dataframe containing the characters with trope

In [90]:
df_inner_char = df_char.merge(df_tropes, how='inner', on='freeID_char_map')

In [97]:
#df_inner_char.drop(columns=["char","actor"],inplace=True)
df_inner_char.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 501 entries, 0 to 500
Data columns (total 14 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   WikiID           501 non-null    object 
 1   freeID           501 non-null    object 
 2   char_name        501 non-null    object 
 3   actor_DOB        499 non-null    object 
 4   actor_gender     500 non-null    object 
 5   actor_height     469 non-null    float64
 6   actor_ethnicity  333 non-null    object 
 7   actor_name       501 non-null    object 
 8   Actor_age        483 non-null    float64
 9   freeID_char_map  501 non-null    object 
 10  FreeID_char      501 non-null    object 
 11  FreeID_actor     501 non-null    object 
 12  trope            501 non-null    object 
 13  movie            501 non-null    object 
dtypes: float64(2), object(12)
memory usage: 74.9+ KB
